In [1]:

import gurobipy as gp
from gurobipy import GRB

In [2]:
A1_cost = 3.25
B1_cost = 3.40
A_r = 100000
B_r = 30000
R = 100000
R_cost = 0.35

#snd article

A_r1= 0.15
A_r2 = 0.85
B_r1 = 0.25
B_r2 = 0.75

Conv = 50000
Conv_c = 0.25
Conv_r1 = 0.5
Conv_r2 = 0.5

Filt_c = 0.10


#sell

p2_c = 3.80
p1_c45 = 5.50
p1_c4 = 5.2
p1_c_inf = 5.0

p1_need = 40000

p1_b = 5.75
p2_b = 4.00



In [3]:
a = gp.Model('smth')
Q1 = a.addVar(vtype = GRB.INTEGER, name = 'buyA_before')
Q2 = a.addVar(vtype = GRB.INTEGER, name = 'buyB_before')
Q3 = a.addVar(vtype = GRB.INTEGER, name = '2to1_convert')
Q4 = a.addVar(vtype = GRB.INTEGER, name = '1for_5.5')
Q5 = a.addVar(vtype = GRB.INTEGER, name = '1for_5.2')
Q6 = a.addVar(vtype = GRB.INTEGER, name = 'buy1_after')
Q7 = a.addVar(vtype = GRB.INTEGER, name = 'buy2_after')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-10


In [4]:

pr1_amount = A_r1*Q1 + B_r1*Q2 + Conv_r1*Q3 + Q6
pr2_amount = A_r2*Q1 + B_r2*Q2 - Conv_r2*Q3 + Q7

charges = A1_cost*Q1 + B1_cost*Q2 + R_cost*(Q1 + Q2) + Conv_c*Q3 +\
+ p1_b*Q6 + p2_b*Q7

income = p1_c45*Q4 + p1_c4*Q5 + p1_c_inf*(pr1_amount - Q4 - Q5)+ p2_c*pr2_amount

In [5]:
a.setObjective(income - charges, GRB.MAXIMIZE)


In [6]:
a.addConstr(Q1<=A_r)
a.addConstr(Q2<=B_r)
a.addConstr(Q1+Q2<=R)
a.addConstr(Q3<=Conv)
a.addConstr(Q4<=45000)
a.addConstr(Q5<=4000)
a.addConstr(pr1_amount>=p1_need)
a.addConstr(pr1_amount - Q4-Q5>=0)
a.addConstr (Q3<=A_r2*Q1 + B_r2*Q2 + Q7)

a.addConstr(Q4==0)
a.addConstr(Q5==0)
a.addConstr(Q2>=0)
a.addConstr(Q3>=0)
a.addConstr(Q6>=0)
a.addConstr(Q7>=0)
a.update()

print(a.display())
a.optimize()



Maximize
<gurobi.LinExpr: 0.38 buyA_before + 0.34999999999999976 buyB_before
+ 0.3500000000000001 2to1_convert + 0.5 1for_5.5 + 0.20000000000000018 1for_5.2 +
-0.75 buy1_after + -0.20000000000000018 buy2_after>
Subject To
  R0: <gurobi.LinExpr: buyA_before> <= 100000
  R1: <gurobi.LinExpr: buyB_before> <= 30000
  R2: <gurobi.LinExpr: buyA_before + buyB_before> <= 100000
  R3: <gurobi.LinExpr: 2to1_convert> <= 50000
  R4: <gurobi.LinExpr: 1for_5.5> <= 45000
  R5: <gurobi.LinExpr: 1for_5.2> <= 4000
R6: <gurobi.LinExpr: 0.15 buyA_before + 0.25 buyB_before + 0.5 2to1_convert +
 buy1_after> >= 40000
R7: <gurobi.LinExpr: 0.15 buyA_before + 0.25 buyB_before + 0.5 2to1_convert + -1.0
 1for_5.5 + -1.0 1for_5.2 + buy1_after> >= 0
R8: <gurobi.LinExpr: -0.85 buyA_before + -0.75 buyB_before + 2to1_convert + -1.0
 buy2_after> <= 0
  R9: <gurobi.LinExpr: 1for_5.5> = 0
  R10: <gurobi.LinExpr: 1for_5.2> = 0
  R11: <gurobi.LinExpr: buyB_before> >= 0
  R12: <gurobi.LinExpr: 2to1_convert> >= 0
  R13: <gur

In [7]:
if a.status == GRB.OPTIMAL:
    print('\nProfit: %g' % a.objVal)
    for v in a.getVars():
        print('%s %g' % (v.varName, v.x))
else:
    print('No solution')


Profit: 55500
buyA_before 100000
buyB_before 0
2to1_convert 50000
1for_5.5 0
1for_5.2 0
buy1_after 0
buy2_after 0
